### Load Packages

In [1]:
import numpy as np
import zipfile
import matplotlib.pyplot as plt
from PIL import Image
import os
import tensorflow as tf
import cv2

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Prepare Data for Training and Testing

In [ ]:
img

In [3]:
# get the number of the image in the file.
# there are 985 image in this file.
directory_name = "/content/drive/MyDrive/EECS-6893-Project/MMU2-Iris-Database"
x, y = np.zeros([1970, 224, 224, 3]), np.zeros([1970])
from collections import defaultdict
index = 0
record = set()
vis = defaultdict(int)
n = 0
count = 0
for filename in os.listdir(directory_name):
  image = Image.open(directory_name + '/' + filename)
  checker = filename[:4]
  if checker == '1000':
    continue


  if not checker in record:
    record.add(checker)
    n += 1
  #print(directory_name + '/' + filename)
  vis[str(checker)] += 1
  image = image.resize((224,224))
  image = np.asarray(image)
  x[index,:,:,:] = image
  y[index] = n

  flip = np.flip(x[index],1)
  flip = np.resize(flip, (1,224,224,3))
  x[index+985,:,:,:] = flip
  y[index+985] = n

  index += 1

In [4]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)
print(len(x_train), len(x_test))


1576 394


In [5]:
index_train = np.random.permutation(len(x_train))
index_test = np.random.permutation(len(x_test))
x_train, y_train = x_train[index_train], y_train[index_train]
x_test, y_test = x_test[index_test], y_test[index_test]

In [6]:
x_train, x_test = tf.cast(x_train, tf.float32)/255.0, tf.cast(x_test, tf.float32)/255.0
y_train, y_test = tf.one_hot(y_train, depth=197), tf.one_hot(y_test, depth=197)
#y_train, y_test = tf.cast(y_train, tf.float32), tf.cast(y_test, tf.float32)

### Model Definition

In [7]:
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras import Model

from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.layers import Flatten

In [8]:
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras import Model

from tensorflow.keras import Model
pre_trained_model = ResNet50(input_shape=(224,224,3), weights='imagenet')
ResNet = Model(inputs=pre_trained_model.input,outputs=pre_trained_model.layers[-2].output)
#ResNet.summary()

# First time, train without adjusting the ResNet weights to train the final dense layer first
ResNet.trainable =  False
model = tf.keras.Sequential([
                             ResNet,
                             tf.keras.layers.Dense(197, activation='softmax')
                                      ])
model.summary()

102981632/102967424 [==============================] - 3s 0us/step
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 model (Functional)          (None, 2048)              23587712  
                                                                 
 dense (Dense)               (None, 197)               403653    
                                                                 
Total params: 23,991,365
Trainable params: 403,653
Non-trainable params: 23,587,712
_________________________________________________________________


In [ ]:
from tensorflow.keras import Model
pre_trained_model = VGG16(input_shape=(224,224,3), weights='imagenet')
vgg_model = Model(inputs=pre_trained_model.input,outputs=pre_trained_model.layers[-2].output)


# First time, train without adjusting the ResNet weights to train the final dense layer first
vgg_model.trainable =  False
model = tf.keras.Sequential([
                             vgg_model,
                             Flatten(),
                             tf.keras.layers.Dense(512, activation='relu'),
                             tf.keras.layers.Dense(197, activation='softmax')
                                      ])
model.summary()

553476096/553467096 [==============================] - 11s 0us/step
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 model_1 (Functional)        (None, 4096)              134260544 
                                                                 
 flatten (Flatten)           (None, 4096)              0         
                                                                 
 dense_1 (Dense)             (None, 512)               2097664   
                                                                 
 dense_2 (Dense)             (None, 197)               101061    
                                                                 
Total params: 136,459,269
Trainable params: 2,198,725
Non-trainable params: 134,260,544
_________________________________________________________________


In [9]:
model.add_loss(lambda: 0.01*tf.reduce_sum(tf.square(model.layers[-1].kernel)))

model.compile(optimizer=tf.keras.optimizers.Adam(0.0002),
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True), 
              metrics=tf.keras.metrics.CategoricalAccuracy())

In [10]:
model.fit(x_train, y_train, batch_size=24, epochs=1, validation_split=0.2)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/dispatch.py:1096: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


53/53 [==============================] - 20s 126ms/step - loss: 8.2609 - categorical_accuracy: 0.0040 - val_loss: 7.4678 - val_categorical_accuracy: 0.0063


In [13]:
#vgg_model.trainable = True
ResNet.trainable =  True
model.add_loss(lambda: 0.01*tf.reduce_sum(tf.square(model.layers[-1].kernel)))

model.compile(optimizer=tf.keras.optimizers.Adam(0.0002),
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False), 
              metrics=tf.keras.metrics.CategoricalAccuracy())

model.fit(x_train, y_train, batch_size=32, epochs=20, validation_split=0.2)

Epoch 1/20
40/40 [==============================] - 16s 227ms/step - loss: 0.5883 - categorical_accuracy: 0.9857 - val_loss: 5.0027 - val_categorical_accuracy: 0.0348
Epoch 2/20
40/40 [==============================] - 7s 187ms/step - loss: 0.4444 - categorical_accuracy: 0.9921 - val_loss: 4.5780 - val_categorical_accuracy: 0.1013
Epoch 3/20
40/40 [==============================] - 8s 189ms/step - loss: 0.3974 - categorical_accuracy: 0.9881 - val_loss: 4.0307 - val_categorical_accuracy: 0.1899
Epoch 4/20
40/40 [==============================] - 7s 187ms/step - loss: 0.3691 - categorical_accuracy: 0.9944 - val_loss: 2.8895 - val_categorical_accuracy: 0.5032
Epoch 5/20
40/40 [==============================] - 7s 187ms/step - loss: 0.3660 - categorical_accuracy: 0.9921 - val_loss: 2.8251 - val_categorical_accuracy: 0.5127
Epoch 6/20
40/40 [==============================] - 8s 188ms/step - loss: 0.3896 - categorical_accuracy: 0.9905 - val_loss: 2.6148 - val_categorical_accuracy: 0.5032
Epo

In [14]:
model.evaluate(x_test, y_test)

13/13 [==============================] - 1s 60ms/step - loss: 0.6924 - categorical_accuracy: 0.9873


[0.6924493312835693, 0.9873096346855164]

In [ ]:
model.save_weights('./MMU2-Iris-Database-weights')

array([[1.61365833e-07, 6.09797510e-08, 5.37834559e-08, ...,
        4.74314021e-07, 1.05221375e-07, 4.89214749e-08],
       [3.40944234e-06, 1.86130399e-06, 1.26375733e-06, ...,
        1.35170058e-06, 1.77589084e-06, 9.16988938e-07],
       [1.81382961e-06, 5.46695310e-06, 1.38406153e-06, ...,
        1.03409832e-06, 3.44551372e-05, 2.01978878e-06],
       ...,
       [1.01617388e-06, 7.90173829e-07, 1.37897507e-06, ...,
        7.09842993e-07, 5.53538166e-06, 5.27412340e-07],
       [2.24078303e-08, 3.71270801e-08, 3.48713449e-08, ...,
        2.77382071e-07, 2.02812870e-07, 3.27055325e-08],
       [1.94871654e-06, 7.16348723e-07, 4.42797585e-07, ...,
        2.34162982e-07, 5.90340790e-07, 4.58690153e-07]], dtype=float32)

In [ ]:
np.mean(np.argmax(y_test, axis=1)==np.argmax(pred, axis=1))

1.0